In [1]:
from __future__ import print_function, division
#Allows relative imports
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports from files
from src.preprocessing import *
from src.VAE_train import *
from vae_cel.vae_cel import *
from clf.clf_train_eval_helpers import * 
from src.embedding_visualisation import * 
from src.loss_metrics import *
from src.pickling import *
from src.datasets import *

import pandas as pd 
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

#checking gpu status
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))
    
#Plot and stuff
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 200
sns.set_style('darkgrid')

torch.cuda.empty_cache()
# Ignore warnings)
import warnings
warnings.filterwarnings("ignore")
    
%load_ext autoreload
%autoreload 2

Using : cuda


### Reloading VAE

In [3]:
from src.torch_util import *
vae = VAE_cel(latent_dim = 100, aa_dim = 25)
vae = load_model(vae, '../output/HyperbolicContinueTraining/LOWERBETA_DirectlyMax/BEST_VAE_tune-weighted0.5_latent100_Pad-before_Annealing-None-gamma1.0.pth.tar')
vae.to(device);

Model succesfully loaded from ../output/HyperbolicContinueTraining/LOWERBETA_DirectlyMax/BEST_VAE_tune-weighted0.5_latent100_Pad-before_Annealing-None-gamma1.0.pth.tar:
	epoch: 74


##### hidden preprocessing cell

In [50]:
# No need to rerun this cell, just read the next
cd_db = pd.read_csv('../training_data_new/db/cd4_cd8_db.csv')
print(len(cd_db.query('v_gene != "unresolved" and j_gene != "unresolved"'))/len(cd_db))
cd_db = cd_db.query('v_gene != "unresolved" and j_gene != "unresolved"')
cd_db = cd_db.query('not v_gene.str.contains("or")')
cd_db['v'] = cd_db['v_gene'].apply(lambda x: int(x.split('V')[1].split('-')[0])-1)
cd_db['j'] = cd_db['j_gene'].apply(lambda x: int(x.split('J')[1].split('-')[0])-1)
cd_db= cd_db.query('cdr3_amino_acid.str.len()<=23')
cd_db.to_csv('../training_data_new/db/cd4_cd8_db.csv', index=False, header=True)

In [18]:
vals = cd_db[['cdr3_amino_acid','v','j']].values
embedding_df = pd.DataFrame(columns = ['z_'+str(i) for i in range(100)])
vae.eval()
vae.to('cuda')
for batch in tqdm(list(chunks(vals, 2**14))):
    b = batch_aa_vj(batch, 23, 0.5, 'before',True, False, 'cuda')
    with torch.no_grad():
        z = vae.embed(b)
    embedding_df = embedding_df.append(pd.DataFrame(data = z.cpu().numpy(), 
                                                    columns = ['z_'+str(i) for i in range(100)]),
                                       ignore_index = True)

  0%|          | 0/154 [00:00<?, ?it/s]

In [4]:
class MLP_binary(nn.Module):
    def __init__(self, n_layers, n_hidden, activation = nn.SELU(), p_drop = 0.5):
        super(MLP_binary, self).__init__()
        if p_drop >0 and p_drop <1:
            self.drop = nn.Dropout(p_drop)
        else: 
            self.drop = nn.Identity()
            
        self.input_layers = nn.Sequential(nn.Linear(100, 256),
                                          nn.BatchNorm1d(256),
                                          activation,
                                          self.drop,
                                          nn.Linear(256, 512),
                                          nn.BatchNorm1d(512),
                                          activation,
                                          self.drop,
                                          nn.Linear(512, n_hidden),
                                          activation,
                                          self.drop)
        layers = []
        for i in range(n_layers):
            layers.append(nn.Linear(n_hidden, n_hidden))
            layers.append(activation)
            layers.append(self.drop) 
        
        self.hidden_layers = nn.Sequential(*layers)
        self.output = nn.Sequential(nn.Linear(n_hidden, n_hidden//2),
                                    nn.BatchNorm1d(n_hidden//2),
                                    activation,
                                    self.drop,
                                    
                                    nn.Linear(n_hidden//2, 10),
                                    activation,
                                    
                                    nn.Linear(10, 1))

    def forward(self, x):
        x = self.input_layers(x)
        x = self.hidden_layers(x)
        x = self.output(x) #No activation because I want to return logits for the BCELoss
        return x.view(-1,)

# VAE: cd8 vs cd4

In [136]:
# Already pre_labeled&embedded & subsetted DF
#df = pd.read_csv('../training_data_new/db/cd4_cd8_db.csv')
# Restricting to spleen only like in the Sonnia paper
subset = df.query('sample_name.str.contains("Spleen") and `cell-type` != "Treg"')
torch.manual_seed(20)

In [137]:
subset = pd.read_csv('../training_data_new/db/cd8_tconv_subset.csv')
subset.head()

,sample_name,cdr3_amino_acid,v_gene,j_gene,cell-type,v,j
0,T1D_6265_Spleen_CD8+,CASSQGDRATYEQYF,TCRBV04-01,TCRBJ02-05,cd8,3,1
1,T1D_6265_Spleen_CD8+,CASSQGPSSETQYF,TCRBV04-01,TCRBJ02-05,cd8,3,1
2,T1D_6265_Spleen_CD8+,CASSQGRGNQPRHF,TCRBV04-01,TCRBJ01-05,cd8,3,0
3,T1D_6265_Spleen_CD8+,CASSQVWDSFTDTQYF,TCRBV04-01,TCRBJ02-03,cd8,3,1
4,T1D_6265_Spleen_CD8+,CASSQGSYNSPLHF,TCRBV04-01,TCRBJ01-06,cd8,3,0


In [138]:
subset.to_csv('../training_data_new/db/cd8_tconv_subset.csv', index=False)
len(subset), subset['cell-type'].unique()

(670315, array(['cd8', 'Tconv'], dtype=object))

In [99]:
from src.torch_util import *
vae = VAE_cel(latent_dim = 100, aa_dim = 25)
vae = load_model(vae, '../output/HyperbolicContinueTraining/LOWERBETA_DirectlyMax/BEST_VAE_tune-weighted0.5_latent100_Pad-before_Annealing-None-gamma1.0.pth.tar')
vae.to(device);
vae.eval()

Model succesfully loaded from ../output/HyperbolicContinueTraining/LOWERBETA_DirectlyMax/BEST_VAE_tune-weighted0.5_latent100_Pad-before_Annealing-None-gamma1.0.pth.tar:
	epoch: 74


In [139]:
aa = subset[['cdr3_amino_acid','v','j']].values
df = pd.DataFrame(columns = ['z_'+str(i) for i in range(100)])
vae.eval()

with torch.no_grad():
    for ch in tqdm(list(chunks(aa, 2**15))):
        x = batch_aa_vj(ch, 23, 0.5, 'before',True,False,'cuda')
        emb = vae.embed(x)
        df = df.append(pd.DataFrame(emb.cpu().numpy(), columns = df.columns), ignore_index=True)
df = df.reset_index().drop(columns = ['index'])
subset = subset.merge(df,left_index =True,right_index=True)#.drop(columns=['index'])
subset.head(), len(subset)

  0%|          | 0/21 [00:00<?, ?it/s]

(            sample_name   cdr3_amino_acid      v_gene      j_gene cell-type  \
 0  T1D_6265_Spleen_CD8+   CASSQGDRATYEQYF  TCRBV04-01  TCRBJ02-05       cd8   
 1  T1D_6265_Spleen_CD8+    CASSQGPSSETQYF  TCRBV04-01  TCRBJ02-05       cd8   
 2  T1D_6265_Spleen_CD8+    CASSQGRGNQPRHF  TCRBV04-01  TCRBJ01-05       cd8   
 3  T1D_6265_Spleen_CD8+  CASSQVWDSFTDTQYF  TCRBV04-01  TCRBJ02-03       cd8   
 4  T1D_6265_Spleen_CD8+    CASSQGSYNSPLHF  TCRBV04-01  TCRBJ01-06       cd8   
 
    v  j       z_0       z_1       z_2  ...      z_90      z_91      z_92  \
 0  3  1 -0.081669 -0.252335 -1.006883  ... -0.954688  1.651380  2.203298   
 1  3  1 -0.282169  0.679195  0.474698  ...  0.554570 -1.690838  1.730452   
 2  3  0  0.049712 -0.012633 -0.832655  ...  0.963157 -1.863461  0.847668   
 3  3  1 -0.979778 -1.109508 -0.356508  ... -1.779394 -2.086255 -2.818264   
 4  3  0  0.958953 -0.986798  0.631857  ...  0.837093 -0.051003  0.784995   
 
        z_93      z_94      z_95      z_96      z_97  

In [140]:
# Split X into Train/Test
subset['label']= subset['cell-type'].apply(lambda x: 1 if x == 'cd8' else 0)
X = subset[['z_'+str(i) for i in range(100)]].values
y = subset['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 20)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))

0.13163027830199234
0.13163214309690222


In [145]:
import random
import xgboost as xgb
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
random.seed(20)
torch.cuda.empty_cache()
clf['xgb'] = {'clf':xgb.XGBClassifier(n_estimators=400,
                                      objective='binary:logistic',
                                      missing=None, seed = 20,
                                      n_jobs = 4, tree_method = 'gpu_hist',
                                      max_depth = 6),
              'roc': 0}

clf['log'] = {'clf': LogisticRegressionCV(10, max_iter = 500,
                                          cv=5, random_state=20, n_jobs = 8),
              'roc': 0}

print('Doing XGB')
clf['xgb']['clf'].fit(X_train, y_train, verbose = True,
                                 early_stopping_rounds = 5, 
                                 eval_metric = 'aucpr',
                                 eval_set = [(X_test, y_test)])

scores = clf['xgb']['clf'].predict_proba(X_test)[:,1]
clf['xgb']['roc'] = roc_auc_score(y_test, scores)
k = 'xgb'
print(f'Classifier : {k}, ROC AUC {clf[k]["roc"]:.3%}')

for name in ['log']:#,'rf']:
    print(f'Doing {name}')
    clf[name]['clf'].fit(X_train,y_train)
    scores = clf[name]['clf'].predict_proba(X_test)[:,1]
    clf[name]['roc'] = roc_auc_score(y_test,scores)
    print(f'Classifier : {name}, ROC AUC {clf[name]["roc"]:.3%}')

Doing XGB
[0]	validation_0-aucpr:0.17971
[1]	validation_0-aucpr:0.18757
[2]	validation_0-aucpr:0.18777
[3]	validation_0-aucpr:0.19009
[4]	validation_0-aucpr:0.19080
[5]	validation_0-aucpr:0.19202
[6]	validation_0-aucpr:0.19482
[7]	validation_0-aucpr:0.19579
[8]	validation_0-aucpr:0.19693
[9]	validation_0-aucpr:0.19694
[10]	validation_0-aucpr:0.19781
[11]	validation_0-aucpr:0.19862
[12]	validation_0-aucpr:0.19884
[13]	validation_0-aucpr:0.19919
[14]	validation_0-aucpr:0.20000
[15]	validation_0-aucpr:0.19990
[16]	validation_0-aucpr:0.20057
[17]	validation_0-aucpr:0.20023
[18]	validation_0-aucpr:0.20002
[19]	validation_0-aucpr:0.20008
[20]	validation_0-aucpr:0.19995
[21]	validation_0-aucpr:0.20025
Classifier : xgb, ROC AUC 62.321%
Doing log
Classifier : log, ROC AUC 55.510%


In [146]:
name = 'rf'
clf['rf'] = {'clf': RandomForestClassifier(n_estimators=300,
                                           min_samples_split = 200,
                                           min_samples_leaf = 600,
                                           verbose=1, max_depth = 8, n_jobs=8),
             'roc': 0}
print(f'Doing {name}')
clf[name]['clf'].fit(X_train,y_train)
scores = clf[name]['clf'].predict_proba(X_test)[:,1]
clf[name]['roc'] = roc_auc_score(y_test,scores)
print(f'Classifier : {name}, ROC AUC {clf[name]["roc"]:.3%}')

Doing rf


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   37.7s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  3.0min
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:  4.7min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s


Classifier : rf, ROC AUC 59.745%


[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.9s finished


In [147]:
save_pkl('../output/classifier_gridsearch/VAE_spleen_cd4cd8_clf_dict.pkl', clf)

In [ ]:
model = MLP_binary()

# AutoEncoder

In [51]:
from src.vautoencoders import AutoEncoder
autoencoder = AutoEncoder(seq_len=23, aa_dim = 21, latent_dim = 40)
autoencoder = load_model(autoencoder, './Best_AE_model_latdim40_withSELU.pth.tar')
autoencoder.to('cuda')
torch.cuda.empty_cache()

Model succesfully loaded from ./Best_AE_model_latdim40_withSELU.pth.tar:
	epoch: 24


In [150]:
xd = subset[subset.columns[0:7]]
xd['label'] = xd.apply(lambda x: 1 if x['cell-type'] == 'cd8' else 0,axis=1)

In [152]:
aa = subset.cdr3_amino_acid.values
df2 = pd.DataFrame(columns = ['lat_'+str(i) for i in range(40)])
autoencoder.to()
autoencoder.eval()
with torch.no_grad():
    for ch in tqdm(list(chunks(aa, 2**15))):
        x = onehot_batch(ch, 23, 1.6, 'after').to('cuda')
        emb = autoencoder.embed(x)
        df2 = df2.append(pd.DataFrame(emb.cpu().numpy(), columns = df2.columns))
df2.reset_index(inplace=True)
df2= df2.drop(columns=['index'])
df2 = xd.merge(df2,left_index =True,right_index=True)

  0%|          | 0/21 [00:00<?, ?it/s]

In [157]:
X = df2[['lat_'+str(i) for i in range(40)]].values
y = df2['label'].values
# Split X into Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 20)
# Split Train into Train/Val
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.8, random_state = 20)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [158]:
import random
import xgboost as xgb
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
random.seed(20)

clf={}
n_estimators = [100, 150, 200, 250]
max_depth = [4,6,8,10]
learning_rate = [0.15, 0.3, 0.55, 0.7]
min_child_weight = [1, 3, 5]
max_delta_step = [0, 1, 3]
lambda_ = [1, 5, 10, 100]
alpha = [0, 1e-3, 1e-1, 5]
tree_method = 'gpu_hist'
torch.cuda.empty_cache()
clf['xgb'] = {'clf':xgb.XGBClassifier(n_estimators=400,
                                      objective='binary:logistic',
                                      missing=None, seed = 20,
                                      n_jobs = 4, tree_method = 'gpu_hist',
                                      max_depth = 6),
              'roc': 0}

clf['log'] = {'clf': LogisticRegressionCV(10, max_iter = 300,
                                          cv=5, random_state=20,
                                         n_jobs = 8),
              'roc': 0}

clf['rf'] = {'clf': RandomForestClassifier(n_estimators=225,
                                           min_samples_split = 200,
                                           min_samples_leaf = 600,
                                           verbose=1, max_depth = 7, n_jobs=2),
             'roc': 0}

print('Doing XGB')
clf['xgb']['clf'].fit(X_train, y_train, verbose = True,
                                 early_stopping_rounds = 5, 
                                 eval_metric = 'aucpr',
                                 eval_set = [(X_test, y_test)])

scores = clf['xgb']['clf'].predict_proba(X_test)[:,1]
clf['xgb']['roc'] = roc_auc_score(y_test, scores)
k = 'xgb'
print(f'Classifier : {k}, ROC AUC {clf[k]["roc"]:.3%}')

for name in ['log']:#,'rf']:
    print(f'Doing {name}')
    clf[name]['clf'].fit(X_train,y_train)
    scores = clf[name]['clf'].predict_proba(X_test)[:,1]
    clf[name]['roc'] = roc_auc_score(y_test,scores)
    print(f'Classifier : {name}, ROC AUC {clf[name]["roc"]:.3%}')

Doing XGB
[0]	validation_0-aucpr:0.15476
[1]	validation_0-aucpr:0.15830
[2]	validation_0-aucpr:0.16153
[3]	validation_0-aucpr:0.16267
[4]	validation_0-aucpr:0.16398
[5]	validation_0-aucpr:0.16415
[6]	validation_0-aucpr:0.16561
[7]	validation_0-aucpr:0.16718
[8]	validation_0-aucpr:0.16831
[9]	validation_0-aucpr:0.16886
[10]	validation_0-aucpr:0.16873
[11]	validation_0-aucpr:0.16905
[12]	validation_0-aucpr:0.16949
[13]	validation_0-aucpr:0.17027
[14]	validation_0-aucpr:0.17050
[15]	validation_0-aucpr:0.17100
[16]	validation_0-aucpr:0.17128
[17]	validation_0-aucpr:0.17270
[18]	validation_0-aucpr:0.17334
[19]	validation_0-aucpr:0.17378
[20]	validation_0-aucpr:0.17391
[21]	validation_0-aucpr:0.17434
[22]	validation_0-aucpr:0.17456
[23]	validation_0-aucpr:0.17485
[24]	validation_0-aucpr:0.17482
[25]	validation_0-aucpr:0.17495
[26]	validation_0-aucpr:0.17549
[27]	validation_0-aucpr:0.17633
[28]	validation_0-aucpr:0.17708
[29]	validation_0-aucpr:0.17750
[30]	validation_0-aucpr:0.17769
[31]	val

In [159]:
name = 'rf'
clf['rf'] = {'clf': RandomForestClassifier(n_estimators=300,
                                           min_samples_split = 200,
                                           min_samples_leaf = 600,
                                           verbose=1, max_depth = 8, n_jobs=8),
             'roc': 0}
print(f'Doing {name}')
clf[name]['clf'].fit(X_train,y_train)
scores = clf[name]['clf'].predict_proba(X_test)[:,1]
clf[name]['roc'] = roc_auc_score(y_test,scores)
print(f'Classifier : {name}, ROC AUC {clf[name]["roc"]:.3%}')

Doing rf


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   22.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:  2.8min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s


Classifier : rf, ROC AUC 57.759%


[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.7s finished


In [ ]:
print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))

# Atchley

In [166]:
X = np.empty((1,23,5))
np.concatenate([X, np.empty((10,23,5))]).shape

(11, 23, 5)

In [178]:
from src.preprocessing import atchley_batch

def get_atchley_values(df, random_state=20):
    torch.manual_seed(random_state)
    sequences = df.cdr3_amino_acid.values
    X = np.empty((0, 23, 5))
    for ch in tqdm(list(chunks(sequences, 2**15))):
        X = np.concatenate([X, atchley_batch(sequences, 23)])
    y = df['label'].values
    return X, y

In [ ]:
class MLP_Flat_Atchley(nn.Module):
    #Reusing an architecture from Deepcat 
    def __init__(self):
        super(MLP_Flat_Atchley, self).__init__()
            
        self.layers = nn.Sequential(nn.Linear(115, 256),
                                    nn.BatchNorm1d(256),
                                    nn.SELU(),
                                    nn.Dropout(0.4),
                                    nn.Linear(256, 512),
                                    nn.BatchNorm1d(512),
                                    nn.SELU(),
                                    nn.Dropout(0.4),
                                    nn.Linear(512, 512),
                                    nn.BatchNorm1d(512),
                                    nn.SELU(),
                                    nn.Dropout(0.4),
                                    nn.Linear(512, 256),
                                    nn.BatchNorm1d(256),
                                    nn.SELU(),
                                    nn.Dropout(0.4),
                                   )
        self.output = nn.Sequential(nn.Linear(256, 64),
                                    nn.SELU(),
                                    nn.Dropout(0.4),
                                    nn.Linear(64, 10),
                                    nn.SELU(),
                                    nn.Dropout(0.4),
                                    nn.Linear(10,1))

    def forward(self, x):
        x = self.layers(x.view(-1,x.shape[1]*x.shape[2]*x.shape[3]))
        x = self.output(x)
        return x.view(-1,)
    
    def reset_params(self):
        i = 0
        for x in self.children():
            if x.__class__.__name__ == 'Sequential':
                for layer in x:
                    if hasattr(layer, 'reset_parameters'):
                        layer.reset_parameters()
                        layer.zero_grad()
                        i += 1
            if hasattr(x, 'reset_parameters'):
                x.reset_parameters()
                x.zero_grad()
                i += 1
        if i >= 1: print('Parameters reset.')
            

In [196]:
X = atchley_batch(subset.cdr3_amino_acid.values,23)
y = subset.label.values
# Split X into Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 20)
# Split Train into Train/Val
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.8, random_state = 20)
del X
del y
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = X_train.reshape(-1, X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(-1, X_test.shape[1]*X_test.shape[2])

In [201]:
import random
import xgboost as xgb
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
random.seed(20)
clf={}
torch.cuda.empty_cache()

clf['xgb'] = {'clf':xgb.XGBClassifier(n_estimators=400,
                                      objective='binary:logistic',
                                      missing=None, seed = 20,
                                      n_jobs = 4, tree_method = 'gpu_hist',
                                      max_depth = 6),
              'roc': 0}

clf['log'] = {'clf': LogisticRegressionCV(10, max_iter = 300,
                                          cv=5, random_state=20,
                                         n_jobs = 8),
              'roc': 0}

clf['rf'] = {'clf': RandomForestClassifier(n_estimators=225,
                                           min_samples_split = 200,
                                           min_samples_leaf = 600,
                                           verbose=1, max_depth = 7, n_jobs=2),
             'roc': 0}

print('Doing XGB')
clf['xgb']['clf'].fit(X_train, y_train, verbose = True,
                                 early_stopping_rounds = 5, 
                                 eval_metric = 'aucpr',
                                 eval_set = [(X_test, y_test)])

scores = clf['xgb']['clf'].predict_proba(X_test)[:,1]
clf['xgb']['roc'] = roc_auc_score(y_test, scores)
k = 'xgb'
print(f'Classifier : {k}, ROC AUC {clf[k]["roc"]:.3%}')

for name in ['log']:#,'rf']:
    print(f'Doing {name}')
    clf[name]['clf'].fit(X_train,y_train)
    scores = clf[name]['clf'].predict_proba(X_test)[:,1]
    clf[name]['roc'] = roc_auc_score(y_test,scores)
    print(f'Classifier : {name}, ROC AUC {clf[name]["roc"]:.3%}')
    
name = 'rf'
clf['rf'] = {'clf': RandomForestClassifier(n_estimators=300,
                                           min_samples_split = 200,
                                           min_samples_leaf = 600,
                                           verbose=1, max_depth = 8, n_jobs=8),
             'roc': 0}
print(f'Doing {name}')
clf[name]['clf'].fit(X_train,y_train)
scores = clf[name]['clf'].predict_proba(X_test)[:,1]
clf[name]['roc'] = roc_auc_score(y_test,scores)
print(f'Classifier : {name}, ROC AUC {clf[name]["roc"]:.3%}')

Doing XGB
[0]	validation_0-aucpr:0.16348
[1]	validation_0-aucpr:0.16861
[2]	validation_0-aucpr:0.17264
[3]	validation_0-aucpr:0.17677
[4]	validation_0-aucpr:0.17833
[5]	validation_0-aucpr:0.18174
[6]	validation_0-aucpr:0.18207
[7]	validation_0-aucpr:0.18624
[8]	validation_0-aucpr:0.18820
[9]	validation_0-aucpr:0.18945
[10]	validation_0-aucpr:0.19008
[11]	validation_0-aucpr:0.19191
[12]	validation_0-aucpr:0.19403
[13]	validation_0-aucpr:0.19478
[14]	validation_0-aucpr:0.19503
[15]	validation_0-aucpr:0.19530
[16]	validation_0-aucpr:0.19578
[17]	validation_0-aucpr:0.19633
[18]	validation_0-aucpr:0.19681
[19]	validation_0-aucpr:0.19759
[20]	validation_0-aucpr:0.19847
[21]	validation_0-aucpr:0.20002
[22]	validation_0-aucpr:0.20000
[23]	validation_0-aucpr:0.20040
[24]	validation_0-aucpr:0.20148
[25]	validation_0-aucpr:0.20202
[26]	validation_0-aucpr:0.20243
[27]	validation_0-aucpr:0.20266
[28]	validation_0-aucpr:0.20315
[29]	validation_0-aucpr:0.20291
[30]	validation_0-aucpr:0.20353
[31]	val

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   13.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:  2.2min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.9s


Classifier : rf, ROC AUC 60.371%


[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    1.6s finished


In [202]:
save_pkl('../output/spleenCD4CD8_clf_atchley.pkl', clf)